# Deploying Machine Learning Models using kubectl
This demo shows how you can interact directly with kubernetes using kubectl to create and manage runtime machine learning models. It uses Minikube as the target Kubernetes cluster.
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Minikube](https://github.com/kubernetes/minikube) version v0.24.0 or greater
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)

Start minikube and ensure custom resource validation is activated and ther is 5G of memory.

In [ ]:
!minikube start --memory=5000 --feature-gates=CustomResourceValidation=true

Install Helm

In [1]:
!helm init

Creating /home/clive/.helm 
Creating /home/clive/.helm/repository 
Creating /home/clive/.helm/repository/cache 
Creating /home/clive/.helm/repository/local 
Creating /home/clive/.helm/plugins 
Creating /home/clive/.helm/starters 
Creating /home/clive/.helm/cache/archive 
Creating /home/clive/.helm/repository/repositories.yaml 
Adding stable repo with URL: https://kubernetes-charts.storage.googleapis.com 
Adding local repo with URL: http://127.0.0.1:8879/charts 
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.
Happy Helming!


Label the node to allow load testing to run on it

In [2]:
!kubectl label nodes `kubectl get nodes -o jsonpath='{.items[0].metadata.name}'` role=locust --overwrite

node "minikube" labeled


## Set up REST and gRPC methods

Install gRPC modules for the prediction protos.

In [3]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I./proto --python_out=./proto --grpc_python_out=./proto ./proto/prediction.proto

Illustration of both REST and gRPC requests. 

In [4]:
import requests
from requests.auth import HTTPBasicAuth
from proto import prediction_pb2
from proto import prediction_pb2_grpc
import grpc
import commands

MINIKUBE_IP=commands.getoutput('minikube ip')

def get_token():
    payload = {'grant_type': 'client_credentials'}
    response = requests.post(
                "http://"+MINIKUBE_IP+":30032/oauth/token",
                auth=HTTPBasicAuth('oauth-key', 'oauth-secret'),
                data=payload)
    token =  response.json()["access_token"]
    return token

def rest_request():
    token = get_token()
    headers = {'Authorization': 'Bearer '+token}
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://"+MINIKUBE_IP+":30032/api/v0.1/predictions",
                headers=headers,
                json=payload)
    print response.text
    
def grpc_request():
    token = get_token()
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = [3,2],
                values = [1.0,1.0,2.0,3.0,4.0,5.0]
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel(MINIKUBE_IP+":30033")
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('oauth_token', token)]
    response = stub.Predict(request=request,metadata=metadata)
    print response


## Start seldon-core

In [5]:
!helm install ../helm-charts/seldon-core --name seldon-core

NAME:   seldon-core
LAST DEPLOYED: Wed Feb 14 18:12:54 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1beta1/Deployment
NAME                    DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-apiserver        1        1        1           0          0s
seldon-cluster-manager  1        1        1           0          0s
redis                   1        1        1           0          0s

==> v1/Service
NAME              TYPE       CLUSTER-IP    EXTERNAL-IP  PORT(S)                        AGE
seldon-apiserver  NodePort   10.106.69.12  <none>       8080:30032/TCP,5000:30033/TCP  0s
redis             ClusterIP  10.110.81.10  <none>       6379/TCP                       0s

==> v1/Pod(related)
NAME                                    READY  STATUS             RESTARTS  AGE
seldon-apiserver-7864874f57-mblb2       0/1    ContainerCreating  0      

Install prometheus and grafana for analytics

In [6]:
!helm install ../helm-charts/seldon-core-analytics --name seldon-core-analytics \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false

NAME:   seldon-core-analytics
LAST DEPLOYED: Wed Feb 14 18:13:02 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Service
NAME                      TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)       AGE
alertmanager              ClusterIP  10.97.66.126    <none>       80/TCP        0s
grafana-prom              NodePort   10.104.145.124  <none>       80:30034/TCP  0s
prometheus-node-exporter  ClusterIP  None            <none>       9100/TCP      0s
prometheus-seldon         ClusterIP  10.103.161.138  <none>       80/TCP        0s

==> v1beta1/DaemonSet
NAME                      DESIRED  CURRENT  READY  UP-TO-DATE  AVAILABLE  NODE SELECTOR  AGE
prometheus-node-exporter  1        1        0      1           0          <none>         0s

==> v1/Pod(related)
NAME                                      READY  STATUS             RESTARTS  AGE
grafana-prom-import-dashboards-9kc9f      0/1    ContainerCreating  0         0s
alertmanager-deployment-5478cc6dbb-kjwlf  0/1    ContainerC

Check all services are running before proceeding.

In [7]:
!kubectl get pods

NAME                                       READY     STATUS              RESTARTS   AGE
alertmanager-deployment-5478cc6dbb-kjwlf   0/1       ContainerCreating   0          4s
grafana-prom-deployment-7b45fb85d4-lt4pq   0/1       ContainerCreating   0          4s
grafana-prom-import-dashboards-9kc9f       0/1       ContainerCreating   0          4s
prometheus-deployment-5db585cb7b-pdhdf     0/1       ContainerCreating   0          4s
prometheus-node-exporter-n4725             0/1       ContainerCreating   0          4s
redis-df886d999-n872b                      0/1       ContainerCreating   0          11s
seldon-apiserver-7864874f57-mblb2          1/1       Running             0          11s
seldon-cluster-manager-576c6ddf5-crrrv     0/1       ContainerCreating   0          11s


# Integrating with Kubernetes API

## Validation

Using OpenAPI Schema certain basic validation can be done before the custom resource is accepted.

In [8]:
!kubectl create -f resources/model_invalid1.json

The SeldonDeployment "seldon-deployment-example" is invalid: []: Invalid value: map[string]interface {}{"apiVersion":"machinelearning.seldon.io/v1alpha1", "kind":"SeldonDeployment", "metadata":map[string]interface {}{"selfLink":"", "clusterName":"", "labels":map[string]interface {}{"app":"seldon"}, "name":"seldon-deployment-example", "namespace":"default", "creationTimestamp":"2018-02-14T18:14:32Z", "uid":"e7f66ff0-11b2-11e8-ac1f-0800278d7f95"}, "spec":map[string]interface {}{"predictors":[]interface {}{map[string]interface {}{"annotations":map[string]interface {}{"predictor_version":"v1"}, "componentSpec":map[string]interface {}{"spec":map[string]interface {}{"containers":[]interface {}{map[string]interface {}{"image":"seldonio/mean_classifier:0.6", "imagePullPolicy":22, "name":"mean-classifier", "resources":map[string]interface {}{"requests":map[string]interface {}{"memory":"1Mi"}}}}, "terminationGracePeriodSeconds":20}}, "graph":map[string]interface {}{"children":[]interface {}{}, "

For the more complex cases, e.g. checking if the graph predictive unit names for models each have an associated container in the pod spec, we need to check inside the custom resource operator and add a FAILED status.

In [ ]:
!kubectl create -f resources/model_invalid2.json

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

In [4]:
!kubectl delete -f resources/model_invalid2.json

seldondeployment "seldon-deployment-example" deleted


## Normal Operation
A simple example is shown below we use a single prepacked model for illustration. The spec contains a set of predictors each of which contains a ***componentSpec*** which is a Kubernetes [PodTemplateSpec](https://kubernetes.io/docs/api-reference/v1.9/#podtemplatespec-v1-core) alongside a ***graph*** which describes how components fit together.

In [9]:
!cat resources/model_tmp.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction",
            "deployment_version": "v1"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpec": {
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/meanclassifier:0.1",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
             

## Create Seldon Deployment

Deploy the runtime graph to kubernetes.

In [12]:
!kubectl apply -f resources/model_tmp.json

seldondeployment "seldon-deployment-example" created


In [13]:
!kubectl get seldondeployments

NAME                        AGE
seldon-deployment-example   22s


In [14]:
!kubectl describe seldondeployments seldon-deployment-example 

Name:         seldon-deployment-example
Namespace:    default
Labels:       app=seldon
Annotations:  kubectl.kubernetes.io/last-applied-configuration={"apiVersion":"machinelearning.seldon.io/v1alpha1","kind":"SeldonDeployment","metadata":{"name":"seldon-deployment-example","namespace":"default","self...
API Version:  machinelearning.seldon.io/v1alpha1
Kind:         SeldonDeployment
Metadata:
  Cluster Name:        
  Creation Timestamp:  2018-02-14T18:34:12Z
  Generation:          0
  Resource Version:    2944
  Self Link:           /apis/machinelearning.seldon.io/v1alpha1/namespaces/default/seldondeployments/seldon-deployment-example
  UID:                 a7392136-11b5-11e8-ac1f-0800278d7f95
Spec:
  Annotations:
    Deployment _ Version:  v1
    Project _ Name:        FX Market Prediction
  Name:                    test-deployment
  Oauth _ Key:             oauth-key
  Oauth _ Secret:          oauth-secret
  Predictors:
    Annotations:
      Predictor _ Version:  v1
    Component Sp

Get the status of the SeldonDeployment. **When ready the replicasAvailable should be 1**.

In [15]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

map[predictorStatus:[map[name:test-deployment-fx-market-predictor replicas:1 replicasAvailable:1]]]

## Get predictions

#### REST Request

In [16]:
rest_request()

{
  "meta": {
    "puid": "2g38ubqjqs591a15oeq86v2to2",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.05133579311531625, 0.12823373759251927]
    }
  }
}


#### gRPC Request

In [17]:
grpc_request()

meta {
  puid: "l7mjf9lkcn92ue9ogcsaso4s93"
}
data {
  names: "proba"
  tensor {
    shape: 3
    shape: 1
    values: 0.128233737593
    values: 0.397314662022
    values: 0.829676081356
  }
}



## Update deployment with canary

We will change the deployment to add a "canary" deployment. This illustrates:
 - Updating a deployment with no downtime
 - Adding an extra predictor to run alongside th exsting predictor.
 
 You could manage different traffic levels by controlling the number of replicas of each.

In [ ]:
!cat resources/model_with_canary.json

In [ ]:
!kubectl apply -f resources/model_with_canary.json

Check the status of the deployments. Note: **Might need to run several times until replicasAvailable is 1 for both predictors**.

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

#### REST Request

In [ ]:
rest_request()

#### gRPC request

In [ ]:
grpc_request()

## Load test

Start a load test which will post REST requests at 10 requests per second.

In [ ]:
!helm install seldon-core-loadtesting --name loadtest  \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --repo https://storage.googleapis.com/seldon-charts

You can view an analytics dashboard inside the cluster at http://192.168.99.100:30034/dashboard/db/prediction-analytics?refresh=5s&orgId=1. Your IP address may be different. get it via minikube ip. Login with:
 - Username : admin
 - password : password (as set when starting seldon-core above)
 
 The dashboard should look like below:
 
 
 <img src="images/dashboard.png" alt="predictor with canary" title="ml graph"/>

# Tear down

In [ ]:
!helm delete loadtest --purge

In [ ]:
!kubectl delete -f resources/model_with_canary.json

In [ ]:
!helm delete seldon-core-analytics --purge

In [ ]:
!helm delete seldon-core --purge